In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/description.txt
/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data_solution.txt
/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data.txt
/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam

2024-03-14 10:38:58.054551: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-14 10:38:58.054624: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-14 10:38:58.055726: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

In [4]:
df = pd.read_csv("/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt",sep=':::', names=['ID', 'TITLE', 'GENRE', 'DESCRIPTION'])
display(df.head())
print(df.shape)

/tmp/ipykernel_900/3152433989.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv("/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt",sep=':::', names=['ID', 'TITLE', 'GENRE', 'DESCRIPTION'])


,ID,TITLE,GENRE,DESCRIPTION
0,1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...
1,2,Cupid (1997),thriller,A brother and sister with a past incestuous r...
2,3,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...
3,4,The Secret Sin (1915),drama,To help their unemployed father make ends mee...
4,5,The Unrecovered (2007),drama,The film's title refers not only to the un-re...


(54214, 4)


import re
def clean_text(text):
    # Remove special characters, punctuation, and symbols
    text = re.sub(r'[^\w\s]', '', text)
    return text

In [5]:
df[0:1]

,ID,TITLE,GENRE,DESCRIPTION
0,1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...


In [6]:
df.drop(columns=['ID','TITLE'],inplace=True)

In [7]:
df.head()

,GENRE,DESCRIPTION
0,drama,Listening in to a conversation between his do...
1,thriller,A brother and sister with a past incestuous r...
2,adult,As the bus empties the students for their fie...
3,drama,To help their unemployed father make ends mee...
4,drama,The film's title refers not only to the un-re...


In [8]:
average_length = df['DESCRIPTION'].str.len().mean()

print(f"The average length of movie descriptions is: {average_length:.2f}")


The average length of movie descriptions is: 600.45


In [9]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [11]:
df['DESCRIPTION'].str.lower()

0         listening in to a conversation between his do...
1         a brother and sister with a past incestuous r...
2         as the bus empties the students for their fie...
3         to help their unemployed father make ends mee...
4         the film's title refers not only to the un-re...
                               ...                        
54209     this short-lived nbc live sitcom centered on ...
54210     the next generation of exploitation. the sist...
54211     ze bestaan echt, is a stand-up comedy about g...
54212     walter and vivian live in the country and hav...
54213     on labor day weekend, 1935, the most intense ...
Name: DESCRIPTION, Length: 54214, dtype: object

In [12]:
import re
def clean_text(text):
    # Remove special characters, punctuation, and symbols
    text = re.sub(r'[^\w\s]', '', text)
    return text
df['DESCRIPTION_1']=df['DESCRIPTION'].apply(clean_text)


In [13]:
from nltk.corpus import stopwords

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    new_text = [word for word in text.split() if word not in stop_words]
    return ' '.join(new_text)
df['DESCRIPTION_1']=df['DESCRIPTION_1'].apply(remove_stopwords)

In [14]:
import spacy 
nlp=spacy.load('en_core_web_sm')
import re 
def tokenize(text):
    tokens=re.split('\W+',text)
    return tokens
df['DESCRIPTION_1']=df['DESCRIPTION_1'].apply(tokenize)
df['DESCRIPTION_1'].head()

0    [Listening, conversation, doctor, parents, 10y...
1    [A, brother, sister, past, incestuous, relatio...
2    [As, bus, empties, students, field, trip, Muse...
3    [To, help, unemployed, father, make, ends, mee...
4    [The, films, title, refers, unrecovered, bodie...
Name: DESCRIPTION_1, dtype: object

In [15]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
def steam_words(text):
    return " ".join([ps.stem(word) for word in text])
df['DESCRIPTION_1'].apply(steam_words)

0        listen convers doctor parent 10yearold oscar l...
1        A brother sister past incestu relationship cur...
2        As bu empti student field trip museum natur hi...
3        To help unemploy father make end meet edith tw...
4        the film titl refer unrecov bodi ground zero a...
                               ...                        
54209    thi shortliv nbc live sitcom center bonino wor...
54210    the next gener exploit the sister kapa bay sor...
54211    Ze bestaan echt standup comedi grow face fear ...
54212    walter vivian live countri difficult time keep...
54213    On labor day weekend 1935 intens hurrican ever...
Name: DESCRIPTION_1, Length: 54214, dtype: object

In [16]:
df.head()

,GENRE,DESCRIPTION,DESCRIPTION_1
0,drama,Listening in to a conversation between his do...,"[Listening, conversation, doctor, parents, 10y..."
1,thriller,A brother and sister with a past incestuous r...,"[A, brother, sister, past, incestuous, relatio..."
2,adult,As the bus empties the students for their fie...,"[As, bus, empties, students, field, trip, Muse..."
3,drama,To help their unemployed father make ends mee...,"[To, help, unemployed, father, make, ends, mee..."
4,drama,The film's title refers not only to the un-re...,"[The, films, title, refers, unrecovered, bodie..."


In [17]:
X=df['DESCRIPTION']
y=df['GENRE']

In [18]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [19]:
X_train

19253     Sex. Betrayal. Seduction. Manipulation. Addic...
12261     Autistic teen, Tony and his friendship with C...
37143     The one-hour special examines this socially, ...
21965     This personal narrative documentary uses my b...
7033      The easy-going village policeman Artturi Saka...
                               ...                        
11284     David Cronenberg (the well-known Canadian dir...
44732     Adan Neumann is a member of a Jewish family t...
38158     Miranda, a lesbian photographer from Guayaqui...
860       They are enemies in their professional lives....
15795     IN 2015 Fashion designer Malan Breton revisit...
Name: DESCRIPTION, Length: 43371, dtype: object

In [20]:
X_test

5089      Catherine St.John-Burke is an independent, up...
30323     Beginning of the century. In a train, a young...
6415      Jerranium 90, a "little rock" that made all t...
20919     Deadly Tales II is an anthology of three horr...
29549     Grammy - music award of the American academy ...
                               ...                        
11342     Four teenage boys come of age in Oslo at the ...
46371     We travel with comedian Michael Blackson arou...
5478      Frédérique is a rich and beautiful woman who ...
23298     A woman drunk dials her senator only to reach...
23605     Suspense and screams abound in this shocking ...
Name: DESCRIPTION, Length: 10843, dtype: object

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=100)  
X_train_vectorized = vectorizer.fit_transform(X_train).toarray()
print(X_train_vectorized)
X_test_vectorized = vectorizer.transform(X_test)

[[0.         0.         0.15753301 ... 0.         0.         0.        ]
 [0.         0.         0.23321958 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.17765169 0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [23]:
from sklearn.preprocessing import LabelEncoder
label_encoder=LabelEncoder()
y_train_label=label_encoder.fit_transform(y_train)
y_test_label=label_encoder.transform(y_test)

In [25]:
model = Sequential([
    Dense(512, activation='relu', input_dim=X_train_vectorized.shape[1]),
    Dropout(0.8),
    Dense(256, activation='relu'),
    # Dropout(0.8),
    Dense(128, activation='relu'),
    # Dropout(0.8),

    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(27, activation='softmax')
])



optimizer = Adam(learning_rate=0.003)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_vectorized, y_train_label, epochs=15, batch_size=64, validation_data=(X_test_vectorized, y_test_label))
# Evaluate the model on the testing dataset
test_loss, test_accuracy = model.evaluate(X_test_vectorized, y_test_label)

print(f'Test Accuracy: {test_accuracy}')
print(f'Test Loss: {test_loss}')

Epoch 1/15
671/678 [============================>.] - ETA: 0s - loss: 2.1444 - accuracy: 0.3992

InvalidArgumentError: {{function_node __wrapped__SerializeManySparse_device_/job:localhost/replica:0/task:0/device:CPU:0}} indices[1] = [0,94] is out of order. Many sparse ops require sorted indices.
    Use `tf.sparse.reorder` to create a correctly ordered copy.

 [Op:SerializeManySparse] name: 